In [1]:
import os
import numpy as np

#from sklearn.metrics.pairwise import cosine_similarity
from numpy import dot
from numpy.linalg import norm

In [2]:
# Load Glove Embedding
gloveFile = "glove.6B/glove.6B.300d.txt"
glove_model = {}

with open(gloveFile, 'r') as f:
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        glove_model[word] = embedding
    print "Done.",len(glove_model)," words loaded!"  

#print glove_model['unk']

Done. 400000  words loaded!


In [3]:
#print glove_model['organization']

In [4]:
# Prepare dataset for NLTK HMM Tagger
# Divide in train and test files [80:20] 

# Directory having content
doc_dir = 'Data/content'

train_file_list = []
test_file_list = []

for f in os.listdir(doc_dir):
    #Random Sampling
    if np.random.uniform(0,1)< 0.8:
        train_file_list.append(f)
    else:
        test_file_list.append(f)

print len(train_file_list)
print len(test_file_list)

1033
274


In [5]:
# Directory having tags
tag_dir = 'Data/new_tags'

tagEmbedding = dict()
tagCount = dict()
win_size = 2
word_dim = 300
#unknown_words = set()

for f in train_file_list:
    #print 'Processing file '+f
    word_file_path = os.path.join(doc_dir, f)
    tag_file_path = os.path.join(tag_dir, f)
    lines_in_word_file = []
    lines_in_tag_file = []
    with open(word_file_path, "rt") as word_file:
        for line in word_file:
            lines_in_word_file.append(line)
    with open(tag_file_path, "rt") as tag_file:
        for line in tag_file:
            lines_in_tag_file.append(line)
    if (len(lines_in_word_file) == len(lines_in_tag_file)) and len(lines_in_word_file) > 0:
        for i in xrange(len(lines_in_word_file)):
            word_in_file = lines_in_word_file[i].split()
            tag_in_file = lines_in_tag_file[i].split()
            length = min(len(word_in_file), len(tag_in_file))
            for j in xrange(length):
                # Get the word embedding of the word
                word_embed = np.zeros(word_dim)
                if (j>(win_size-1)) and (j<length-win_size):
                    replace_unk = 'unk'
                    for k in xrange(2*win_size + 1):
                        if word_in_file[j-win_size+k].lower() in glove_model:
                            word_embed = np.concatenate((word_embed, glove_model[word_in_file[j-win_size+k].lower()]), axis=0)
                        else:
                            #unknown_words.add(word_in_file[j-win_size+k])
                            #word_embed = np.concatenate((word_embed, glove_model['unk']), axis = 0)  
                            word_embed = np.concatenate((word_embed, glove_model[replace_unk]), axis = 0)  
                    # Create embdedding of Tag    
                    if tag_in_file[j] in tagEmbedding:
                        tagEmbedding[tag_in_file[j]] = np.add(tagEmbedding[tag_in_file[j]], word_embed)
                        tagCount[tag_in_file[j]] = tagCount[tag_in_file[j]] + 1
                    else:
                        tagEmbedding[tag_in_file[j]] = word_embed
                        tagCount[tag_in_file[j]] = 1
                    

In [6]:
for key in tagCount:
    tagEmbedding[key] = tagEmbedding[key]/tagCount[key]


In [7]:
# Test
actual_tag_count = dict()
matched_tag_count = dict()
matched_tag_count['PER'] = 0
actual_tag_count['PER'] = 0
matched_tag_count['ORG'] = 0
actual_tag_count['ORG'] = 0
matched_tag_count['LOC'] = 0
actual_tag_count['LOC'] = 0

for f in test_file_list:
    word_file_path = os.path.join(doc_dir, f)
    tag_file_path = os.path.join(tag_dir, f)
    
    lines_in_word_file = []
    lines_in_tag_file = []
    
    with open(word_file_path, "rt") as word_file:
        for line in word_file:
            lines_in_word_file.append(line)
    with open(tag_file_path, "rt") as tag_file:
        for line in tag_file:
            lines_in_tag_file.append(line)
    if (len(lines_in_word_file) == len(lines_in_tag_file)) and len(lines_in_word_file) > 0:
        for i in xrange(len(lines_in_word_file)):
            word_in_file = lines_in_word_file[i].split()
            tag_in_file = lines_in_tag_file[i].split()
            length = min(len(word_in_file), len(tag_in_file))
            for j in xrange(length):
                # Get the word embedding of the word
                word_embed = np.zeros(word_dim)
                if (j>(win_size-1)) and (j<length-win_size):
                    replace_unk = 'unk'
                    
                    for k in xrange(2*win_size + 1):
                        if word_in_file[j-win_size+k].lower() in glove_model:
                            word_embed = np.concatenate((word_embed, glove_model[word_in_file[j-win_size+k].lower()]), axis = 0)
                        else:
                            word_embed = np.concatenate((word_embed, glove_model['unk']), axis=0)  
                    
                    actual_tag = tag_in_file[j][:3]
                    if actual_tag in actual_tag_count.keys():
                        actual_tag_count[actual_tag] = actual_tag_count[actual_tag] + 1
                    # Get the predicted tag
                    nearestCosineDist = 2
                    predicted_tag = ""
                    for tag in tagEmbedding:
                        dist = 1 - (dot(tagEmbedding[tag], word_embed)/(norm(tagEmbedding[tag]) * norm(word_embed)))
                        if dist < nearestCosineDist:
                            predicted_tag = tag[:3]
                            nearestCosineDist = dist
                    if actual_tag == predicted_tag:
                        if actual_tag in matched_tag_count.keys():
                            matched_tag_count[actual_tag] = matched_tag_count[actual_tag] + 1
                        

In [8]:
print('======= Accuracy Class Wise =====================\n')

print('Class    Matched Total %')
print('--------------------------')
print ('PER: '+str(matched_tag_count['PER'])+ ' ' + str(actual_tag_count['PER'])+ ' ' + str(matched_tag_count['PER']*100/actual_tag_count['PER'])+'%')
print ('ORG: '+str(matched_tag_count['ORG']) + ' '+ str(actual_tag_count['ORG'])+ ' '+str(matched_tag_count['ORG']*100/actual_tag_count['ORG'])+'%')
print ('LOC: '+str(matched_tag_count['LOC'])+ ' ' + str(actual_tag_count['LOC'])+ ' '+str(matched_tag_count['LOC']*100/actual_tag_count['LOC'])+'%')

======= Accuracy Class Wise =====================

Class    Matched Total %
--------------------------
PER: 2868 3334 86%
ORG: 1936 3406 56%
LOC: 2693 3749 71%
